In [6]:
import MetaTrader5 as mt5
from backtesting import Backtest, Strategy
import pandas as pd
from datetime import datetime
import pytz
import matplotlib.pyplot as plt

# Function to calculate SL and TP prices
def calculate_prices(entry_price, risk_reward_ratio, order_type, mean_candle_size):
    risk_part, reward_part = map(int, risk_reward_ratio.split(':'))
    risk_amount = mean_candle_size * risk_part
    reward_amount = mean_candle_size * reward_part
    
    if order_type == 'buy':
        sl_price = entry_price - risk_amount
        tp_price = entry_price + reward_amount
    elif order_type == 'sell':
        sl_price = entry_price + risk_amount
        tp_price = entry_price - reward_amount
    else:
        raise ValueError("order_type must be either 'buy' or 'sell'")
    
    return sl_price, tp_price


# Set timezone to UTC
utc_tz = pytz.utc

# Initialize MT5 connection
if not mt5.initialize():
    print("Failed to initialize MT5")
    exit()

login = 51708234
password = "4bM&wuVJcBTnjV"
server = "ICMarketsEU-Demo"
if not mt5.login(login, password, server):
    print("Failed to login to MT5")
    mt5.shutdown()
    exit()

symbol = "EURUSD"
mean_candle_size = 0.010
risk_reward_ratio = "2:3"

timeframe = mt5.TIMEFRAME_D1
start_date = datetime(2023, 11, 27, 0, 0, 0, tzinfo=utc_tz)
end_date = datetime(2024, 9, 25, 0, 0, 0, tzinfo=utc_tz)

# Retrieve OHLC data from MetaTrader
rates = mt5.copy_rates_range(symbol, timeframe, start_date, end_date)
if rates is None:
    print(f"Failed to retrieve data for {symbol}")
    mt5.shutdown()
    exit()

ohlc_data = pd.DataFrame(rates)
ohlc_data['time'] = pd.to_datetime(ohlc_data['time'], unit='s')
ohlc_data.set_index('time', inplace=True)

# Select only the required columns
df = ohlc_data[['open', 'high', 'low', 'close']]
df.columns = ['Open', 'High', 'Low', 'Close']

# Load only available buy or sell predictions from CSV
try:
    df_predbuy = pd.read_csv(f'predict_EURUSD_D1_3112_Buy.csv', index_col=0)
    df_predbuy.index = pd.to_datetime(df_predbuy.index)
except FileNotFoundError:
    df_predbuy = None  # If buy prediction CSV is not found

try:
    df_predsell = pd.read_csv(f'predict_GBPUSD_D1_3112_Sell.csv', index_col=0)
    df_predsell.index = pd.to_datetime(df_predsell.index)
except FileNotFoundError:
    df_predsell = None  # If sell prediction CSV is not found

# Define a strategy based on available buy or sell predictions
class SimpleStrategy(Strategy):
    def init(self):
        self.mean_candle_size = mean_candle_size
        self.risk_reward_ratio = risk_reward_ratio

    def next(self):
        current_time = self.data.index[-1]

        # Check if there's already an open position
        if self.position:
            return

        # Initialize buy and sell prediction to 0 (no trade by default)
        buy_prediction = 0
        sell_prediction = 0

        # Check if buy prediction data exists and use it
        if df_predbuy is not None and current_time in df_predbuy.index:
            buy_prediction = df_predbuy.loc[current_time, 'prediction']

        # Check if sell prediction data exists and use it
        if df_predsell is not None and current_time in df_predsell.index:
            sell_prediction = df_predsell.loc[current_time, 'prediction']

        # Handle Buy Signal
        if buy_prediction == 1:
            entry_price = self.data.Close[-1]
            order_type = 'buy'
            sl_price, tp_price = calculate_prices(entry_price, self.risk_reward_ratio, order_type, self.mean_candle_size)
            self.buy(size=10000, sl=sl_price, tp=tp_price)

        # Handle Sell Signal
        elif sell_prediction == 1:
            entry_price = self.data.Close[-1]
            order_type = 'sell'
            sl_price, tp_price = calculate_prices(entry_price, self.risk_reward_ratio, order_type, self.mean_candle_size)
            self.sell(size=1000, sl=sl_price, tp=tp_price)

# Create and run backtest with the SimpleStrategy
bt = Backtest(df, SimpleStrategy, cash=10000, commission=0.0003, margin=0.02)
output = bt.run()
print(f"Results for {symbol}:")
print(output)
from bokeh.io import output_file, show

# Plot the results using Matplotlib
show(bt.plot())

# Shutdown MT5 connection after data retrieval
mt5.shutdown()


Results for EURUSD:
Start                     2023-11-27 00:00:00
End                       2024-09-25 00:00:00
Duration                    303 days 00:00:00
Exposure Time [%]                   91.666667
Equity Final [$]                  10143.13153
Equity Peak [$]                   10155.83153
Return [%]                           1.431315
Buy & Hold Return [%]                 1.63047
Return (Ann.) [%]                    1.671852
Volatility (Ann.) [%]                5.586634
Sharpe Ratio                         0.299259
Sortino Ratio                        0.433288
Calmar Ratio                         0.342335
Max. Drawdown [%]                   -4.883673
Avg. Drawdown [%]                   -2.357619
Max. Drawdown Duration      240 days 00:00:00
Avg. Drawdown Duration       92 days 00:00:00
# Trades                                    3
Win Rate [%]                        66.666667
Best Trade [%]                       2.776796
Worst Trade [%]                     -1.847214
Avg. Trade [%]

c:\ProgramData\anaconda3\envs\forex_env\Lib\site-packages\backtesting\_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%d %b'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],
c:\ProgramData\anaconda3\envs\forex_env\Lib\site-packages\backtesting\_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%m/%Y'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],
c:\ProgramData\anaconda3\envs\forex_env\Lib\site-packages\backtesting\_plotting.py:456: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  .resample(resample_rule, label='left')
c:\ProgramData\anaconda3\envs\forex_env\Lib\site-packages\backtesting\_plotting.py:659: UserWarning: found multiple competing values for 'toolbar.active_drag' property; using the latest value
  fig = gridplot(
c:\ProgramData\anaconda3\envs\forex_env\Lib\site-packages\

True

In [5]:
# Load predictions from CSV file
df_predsell = pd.read_csv(f'predictUSDCAD_D1Sell copy.csv', index_col=0)
df_predsell.index = pd.to_datetime(df_predsell.index)

# Shift predictions by 1 period forward to simulate real-world decision-making
df_predsell['prediction'] = df_predsell['prediction'].shift(1)

# Define a simple strategy based on sell predictions
class SimpleSellStrategy(Strategy):
    def init(self):
        self.mean_candle_size = mean_candle_size
        self.risk_reward_ratio = risk_reward_ratio

    def next(self):
        current_time = self.data.index[-1]

        # Find the closest match or exactly matching timestamp
        if current_time in df_predsell.index:
            sell_prediction = df_predsell.loc[current_time, 'prediction']
            if isinstance(sell_prediction, pd.Series):
                sell_prediction = sell_prediction.iloc[0]  # Take the first value if multiple exist
        else:
            sell_prediction = 0

        # Execute a sell order if the prediction indicates 1 (signal to sell)
        if sell_prediction == 1:
            entry_price = self.data.Close[-1]
            order_type = 'sell'
            sl_price, tp_price = calculate_prices(entry_price, self.risk_reward_ratio, order_type, self.mean_candle_size)
            self.sell(size=1000, sl=sl_price, tp=tp_price)

# Create and run backtest with the SimpleSellStrategy
bt = Backtest(df, SimpleSellStrategy, cash=10000, commission=0.0003, margin=0.02)
output = bt.run()
print(f"Results for {symbol}:")
print(output)

from bokeh.io import output_file, show

# Plot the results using Bokeh
show(bt.plot())

# Shutdown MT5 connection after data retrieval
mt5.shutdown()


FileNotFoundError: [Errno 2] No such file or directory: 'predictUSDCAD_D1Sell copy.csv'

In [14]:
import MetaTrader5 as mt5
from backtesting import Backtest, Strategy
import pandas as pd
from datetime import datetime
import pytz
import matplotlib.pyplot as plt
import talib

# Function to calculate SL and TP prices
def calculate_prices(entry_price, risk_reward_ratio, order_type, mean_candle_size):
    risk_part, reward_part = map(int, risk_reward_ratio.split(':'))
    risk_amount = mean_candle_size * risk_part
    reward_amount = mean_candle_size * reward_part
    
    if order_type == 'buy':
        sl_price = entry_price - risk_amount
        tp_price = entry_price + reward_amount
    elif order_type == 'sell':
        sl_price = entry_price + risk_amount
        tp_price = entry_price - reward_amount
    else:
        raise ValueError("order_type must be either 'buy' or 'sell'")
    
    return sl_price, tp_price


# Set timezone to UTC
utc_tz = pytz.utc

# Initialize MT5 connection
if not mt5.initialize():
    print("Failed to initialize MT5")
    exit()

login = 51708234
password = "4bM&wuVJcBTnjV"
server = "ICMarketsEU-Demo"
if not mt5.login(login, password, server):
    print("Failed to login to MT5")
    mt5.shutdown()
    exit()

symbol = "USDCAD"
mean_candle_size = 0.0099
risk_reward_ratio = "1:2"

timeframe = mt5.TIMEFRAME_D1
start_date = datetime(2023, 11, 27, 0, 0, 0, tzinfo=utc_tz)
end_date = datetime(2024, 9, 25, 0, 0, 0, tzinfo=utc_tz)

# Retrieve OHLC data from MetaTrader
rates = mt5.copy_rates_range(symbol, timeframe, start_date, end_date)
if rates is None:
    print(f"Failed to retrieve data for {symbol}")
    mt5.shutdown()
    exit()

ohlc_data = pd.DataFrame(rates)
ohlc_data['time'] = pd.to_datetime(ohlc_data['time'], unit='s')
ohlc_data.set_index('time', inplace=True)

# Select only the required columns
df = ohlc_data[['open', 'high', 'low', 'close']]
df.columns = ['Open', 'High', 'Low', 'Close']

# Load predictions from CSV file
df_predsell = pd.read_csv(f'predictUSDCAD_D1Sell copy.csv', index_col=0)
df_predsell.index = pd.to_datetime(df_predsell.index)

# Shift predictions by 1 period to prevent look-ahead bias
df_predsell['prediction'] = df_predsell['prediction'].shift(1)

# Define a filtered strategy based on sell predictions with RSI as a filter
class FilteredSellStrategy(Strategy):
    def init(self):
        self.mean_candle_size = mean_candle_size
        self.risk_reward_ratio = risk_reward_ratio
        self.rsi = self.I(talib.RSI, self.data.Close, 14)
        self.ma = self.I(talib.SMA, self.data.Close, 50)  # 50-period moving average for trend

    def next(self):
        current_time = self.data.index[-1]

        # Check for sell prediction
        if current_time in df_predsell.index:
            sell_prediction = df_predsell.loc[current_time, 'prediction']
            if isinstance(sell_prediction, pd.Series):
                sell_prediction = sell_prediction.iloc[0]
        else:
            sell_prediction = 0

        # Logging for debugging
        print(f"Date: {current_time}, Sell Prediction: {sell_prediction}, RSI: {self.rsi[-1]}, MA: {self.ma[-1]}, Close: {self.data.Close[-1]}")

        # Relaxed condition: RSI > 60 and price below 50-period MA
        if sell_prediction == 1 and self.rsi[-1] > 60 and self.data.Close[-1] < self.ma[-1]:
            entry_price = self.data.Close[-1]
            order_type = 'sell'
            sl_price, tp_price = calculate_prices(entry_price, self.risk_reward_ratio, order_type, self.mean_candle_size)
            self.sell(size=100, sl=sl_price, tp=tp_price)

# Create and run backtest with the FilteredSellStrategy
bt = Backtest(df, FilteredSellStrategy, cash=1000, commission=0.0003, margin=0.02)
output = bt.run()
print(f"Results for {symbol}:")
print(output)

# Shutdown MT5 connection after data retrieval
mt5.shutdown()




Date: 2019-07-03 00:00:00, Sell Prediction: 0.0, RSI: 29.578268967732903, MA: 1.336668, Close: 1.30588
Date: 2019-07-04 00:00:00, Sell Prediction: 1.0, RSI: 28.81837787024199, MA: 1.3357911999999998, Close: 1.3046
Date: 2019-07-05 00:00:00, Sell Prediction: 1.0, RSI: 32.75721736881183, MA: 1.3350216000000001, Close: 1.30731
Date: 2019-07-08 00:00:00, Sell Prediction: 1.0, RSI: 36.101356895691104, MA: 1.3343010000000002, Close: 1.30969
Date: 2019-07-09 00:00:00, Sell Prediction: 1.0, RSI: 40.1549588785247, MA: 1.3337764, Close: 1.3127
Date: 2019-07-10 00:00:00, Sell Prediction: 1.0, RSI: 36.33647118646071, MA: 1.3330658000000002, Close: 1.30807
Date: 2019-07-11 00:00:00, Sell Prediction: 1.0, RSI: 35.358864365373876, MA: 1.3322556000000003, Close: 1.30682
Date: 2019-07-12 00:00:00, Sell Prediction: 1.0, RSI: 32.351823804290305, MA: 1.3314686, Close: 1.30281
Date: 2019-07-15 00:00:00, Sell Prediction: 1.0, RSI: 35.43382385486593, MA: 1.3306706, Close: 1.3049
Date: 2019-07-16 00:00:00, Se

True

In [2]:
import MetaTrader5 as mt5
from backtesting import Backtest, Strategy
import pandas as pd
from datetime import datetime
import pytz
import matplotlib.pyplot as plt

# Function to calculate SL and TP prices
def calculate_prices(entry_price, risk_reward_ratio, order_type, mean_candle_size):
    risk_part, reward_part = map(int, risk_reward_ratio.split(':'))
    risk_amount = mean_candle_size * risk_part
    reward_amount = mean_candle_size * reward_part
    
    if order_type == 'buy':
        sl_price = entry_price - risk_amount
        tp_price = entry_price + reward_amount
    elif order_type == 'sell':
        sl_price = entry_price + risk_amount
        tp_price = entry_price - reward_amount
    else:
        raise ValueError("order_type must be either 'buy' or 'sell'")
    
    return sl_price, tp_price

# Set timezone to UTC
utc_tz = pytz.utc

# Initialize MT5 connection
if not mt5.initialize():
    print("Failed to initialize MT5")
    exit()

# Login details
login = 51708234
password = "4bM&wuVJcBTnjV"
server = "ICMarketsEU-Demo"
if not mt5.login(login, password, server):
    print("Failed to login to MT5")
    mt5.shutdown()
    exit()

symbol = "EURUSD"
mean_candle_size = 0.010  # Example value for mean candle size
risk_reward_ratio = "2:3"  # Example risk-reward ratio

timeframe = mt5.TIMEFRAME_D1
start_date = datetime(2023, 11, 27, 0, 0, 0, tzinfo=utc_tz)
end_date = datetime(2024, 9, 25, 0, 0, 0, tzinfo=utc_tz)

# Retrieve OHLC data from MetaTrader
rates = mt5.copy_rates_range(symbol, timeframe, start_date, end_date)
if rates is None:
    print(f"Failed to retrieve data for {symbol}")
    mt5.shutdown()
    exit()

ohlc_data = pd.DataFrame(rates)
ohlc_data['time'] = pd.to_datetime(ohlc_data['time'], unit='s')
ohlc_data.set_index('time', inplace=True)

# Select only the required columns
df = ohlc_data[['open', 'high', 'low', 'close']]
df.columns = ['Open', 'High', 'Low', 'Close']

# Load prediction data from your CSV
try:
    df_predbuy = pd.read_csv('predict_EURUSD_D1_3112_buy.csv', parse_dates=['time'], index_col='time')
    df_predbuy.index = pd.to_datetime(df_predbuy.index).tz_localize('UTC')
except FileNotFoundError:
    print("Prediction file not found.")
    mt5.shutdown()
    exit()

# Align prediction data with OHLC data
df_predbuy = df_predbuy[df_predbuy.index.isin(df.index)]

# Define a strategy based on buy predictions
class SimpleStrategy(Strategy):
    def init(self):
        self.mean_candle_size = mean_candle_size
        self.risk_reward_ratio = risk_reward_ratio

    def next(self):
        current_time = self.data.index[-1]

        # Check if there's already an open position
        if self.position:
            return

        # Buy prediction logic (simplified)
        if df_predbuy is not None and current_time in df_predbuy.index:
            buy_prediction = df_predbuy.loc[current_time, 'prediction']
            
            # Execute buy if prediction is 1
            if buy_prediction == 1:
                entry_price = self.data.Close[-1]
                order_type = 'buy'
                sl_price, tp_price = calculate_prices(entry_price, self.risk_reward_ratio, order_type, self.mean_candle_size)
                self.buy(size=10000, sl=sl_price, tp=tp_price)

# Create and run backtest with the SimpleStrategy
bt = Backtest(df, SimpleStrategy, cash=10000, commission=0.0003, margin=0.02)
output = bt.run()
print(f"Results for {symbol}:")
print(output)

# Plot the results using Bokeh
from bokeh.io import show
show(bt.plot())

# Shutdown MT5 connection after data retrieval
mt5.shutdown()


Results for EURUSD:
Start                     2023-11-27 00:00:00
End                       2024-09-25 00:00:00
Duration                    303 days 00:00:00
Exposure Time [%]                         0.0
Equity Final [$]                      10000.0
Equity Peak [$]                       10000.0
Return [%]                                0.0
Buy & Hold Return [%]                 1.63047
Return (Ann.) [%]                         0.0
Volatility (Ann.) [%]                     0.0
Sharpe Ratio                              NaN
Sortino Ratio                             NaN
Calmar Ratio                              NaN
Max. Drawdown [%]                        -0.0
Avg. Drawdown [%]                         NaN
Max. Drawdown Duration                    NaN
Avg. Drawdown Duration                    NaN
# Trades                                    0
Win Rate [%]                              NaN
Best Trade [%]                            NaN
Worst Trade [%]                           NaN
Avg. Trade [%]

c:\ProgramData\anaconda3\envs\forex_env\Lib\site-packages\backtesting\_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%d %b'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],
c:\ProgramData\anaconda3\envs\forex_env\Lib\site-packages\backtesting\_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%m/%Y'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],
c:\ProgramData\anaconda3\envs\forex_env\Lib\site-packages\backtesting\_plotting.py:456: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  .resample(resample_rule, label='left')


True